In [1]:
# 📦 Golden Recursive Attractor Deep-Dive: Digit Residue Analysis & Symbolic Residue Discovery
# 🔧 All in one: Colab-ready, overflow-safe, fully self-contained

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sympy import primerange
from IPython.display import display

# === CORE CONSTANTS ===
phi_f = (1 + math.sqrt(5)) / 2
alpha_inv = 137.035999
alpha = 1 / alpha_inv
fine_structure_decimal = 0.0072973525693
delta = math.pi - 3.141412303134

# More constants to test
feigenbaum_delta = 4.669201609
feigenbaum_alpha = 2.502907875
euler_gamma = 0.5772156649
zeta_3 = 1.2020569031

# Fibonacci word decimal
def fibonacci_word(n):
    a, b = "0", "01"
    for _ in range(n):
        a, b = b, b + a
    return b[:2**n]

fib_word_str = fibonacci_word(10)
fib_word_decimal = sum([int(b)*(0.5**(i+1)) for i,b in enumerate(fib_word_str)])
fib_word_decimal = fib_word_decimal if fib_word_decimal != 0 else 1e-10

constants_to_test = {
    "alpha": alpha,
    "fine_structure": fine_structure_decimal,
    "alpha_inv": alpha_inv,
    "pi": math.pi,
    "pi/2": math.pi / 2,
    "pi/3": math.pi / 3,
    "pi/4": math.pi / 4,
    "pi-3": math.pi - 3,
    "sqrt(pi)": math.sqrt(math.pi),
    "pi/sqrt(3)": math.pi / math.sqrt(3),
    "phi": phi_f,
    "phi-1": phi_f - 1,
    "phi^2": phi_f**2,
    "sqrt(phi-1)": math.sqrt(phi_f - 1),
    "sqrt(phi)": math.sqrt(phi_f),
    "1-(phi-1)": 1 - (phi_f - 1),
    "1/phi": 1 / phi_f,
    "1/(phi-1)": 1 / (phi_f - 1),
    "sqrt(2)": math.sqrt(2),
    "sqrt(sqrt(2))": math.sqrt(math.sqrt(2)),
    "1/sqrt(2)": 1 / math.sqrt(2),
    "1/sqrt(sqrt(2))": 1 / math.sqrt(math.sqrt(2)),
    "sqrt(3)": math.sqrt(3),
    "sqrt(sqrt(3))": math.sqrt(math.sqrt(3)),
    "1/sqrt(3)": 1 / math.sqrt(3),
    "1/sqrt(sqrt(3))": 1 / math.sqrt(math.sqrt(3)),
    "feigenbaum_delta": feigenbaum_delta,
    "feigenbaum_alpha": feigenbaum_alpha,
    "euler_gamma": euler_gamma,
    "zeta_3": zeta_3,
    "fib_word_decimal": fib_word_decimal,
    "1/fib_word_decimal": 1 / fib_word_decimal,
    "1 - fib_word_decimal": 1 - fib_word_decimal
}

# === FUNCTIONS ===

# Overflow-safe zeta computation
def golden_damped_zeta_safe(N=50, log_term_cutoff=-700, variant='phi^n'):
    primes = list(primerange(1, 1000))[:N]
    log_phi = math.log(phi_f)
    terms = []

    for i, p in enumerate(primes):
        try:
            n = i + 1
            log_p = math.log(p)

            if variant == 'phi^n':
                exp_term = math.exp(n * log_phi)
            elif variant == 'n^phi':
                exp_term = math.exp(phi_f * math.log(n))
            elif variant == 'n*log(phi)':
                exp_term = n * log_phi
            else:
                raise ValueError("Invalid variant")

            log_term = -exp_term * log_p
            terms.append(math.exp(log_term) if log_term > log_term_cutoff else 0)

        except Exception:
            terms.append(0)

    zeta_val = sum(terms)
    return zeta_val, zeta_val * alpha_inv

# Digit match score
def digit_match_score(a, b, window=6):
    a_str = f"{a:.16f}".replace('.', '')
    b_str = f"{b:.16f}".replace('.', '')
    max_score = 0
    for i in range(len(a_str)-window):
        a_sub = a_str[i:i+window]
        for j in range(len(b_str)-window):
            b_sub = b_str[j:j+window]
            score = sum(x==y for x, y in zip(a_sub, b_sub))
            max_score = max(max_score, score)
    return max_score

# Digit residue match breakdown
def digit_residue_map(a, b, digits=12):
    a_str = f"{a:.16f}".replace('.', '')[:digits]
    b_str = f"{b:.16f}".replace('.', '')[:digits]
    return ''.join(['✔' if x==y else '✗' for x, y in zip(a_str, b_str)])

# Symbolic residue search
def best_matching_constant(N=50, window=6):
    _, scaled_zeta = golden_damped_zeta_safe(N)
    results = []
    for name, value in constants_to_test.items():
        score = digit_match_score(scaled_zeta, value, window=window)
        residue = digit_residue_map(scaled_zeta, value)
        results.append((name, value, score, residue))
    df = pd.DataFrame(results, columns=["Constant", "Value", f"MatchScore (Zeta, {window})", "DigitResidueMap"])
    return df.sort_values(f"MatchScore (Zeta, {window})", ascending=False)

# === RUN ALL ===

# Full symbolic residue report
final_df = best_matching_constant(N=50, window=6)
display(final_df.style.set_properties(**{'font-family': 'monospace'}).background_gradient(subset=[final_df.columns[2]], cmap='YlGnBu'))


,Constant,Value,"MatchScore (Zeta, 6)",DigitResidueMap
0,alpha,0.007297,5,✗✗✗✗✗✗✔✗✗✗✗✔
1,fine_structure,0.007297,5,✗✗✗✗✗✗✔✗✗✗✗✗
5,pi/3,1.047198,4,✗✗✗✗✗✗✔✔✗✗✔✗
25,1/sqrt(sqrt(3)),0.759836,4,✗✗✔✗✗✔✗✗✗✗✗✗
21,1/sqrt(sqrt(2)),0.840896,4,✗✗✗✗✗✗✗✗✗✗✗✗
7,pi-3,0.141593,3,✗✗✗✔✔✗✗✗✗✗✗✗
8,sqrt(pi),1.772454,3,✗✗✗✗✗✗✗✗✗✗✗✗
6,pi/4,0.785398,3,✗✗✗✗✗✗✗✗✗✗✗✗
3,pi,3.141593,3,✗✗✗✔✔✗✗✗✗✗✗✗
20,1/sqrt(2),0.707107,3,✗✗✗✗✗✗✗✗✗✗✔✗
